In [ ]:
cd /home/schlecker/repos/planeteScripts

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.optimize import curve_fit

import output
import plots
import stats
import utils

%load_ext autoreload
%autoreload 2

# default output folder
outputs = '/home/schlecker/phd/planete/outputs/pop06_MstarGrid/'
# outputs = '/media/martin/Daten/phd/planete/outputs/pop06_MstarGrid/'

pd.set_option('display.max_columns', 500)

## Basic statistics

### initiate the 'master' population with subpopulations for each stellar mass
All ref_red data will be in one multiindex-dataframe

In [ ]:
filenames = ['ref_red5e9_0.3Msol.dat', 'ref_red5e9_0.5Msol.dat', 
             'ref_red5e9_0.7Msol.dat', 'ref_red5e9_1.0Msol.dat'] 
fileList = [outputs + f for f in filenames]
Mgrid = output.Population(fileList, name='Mgrid')

In [ ]:
Mgrid.get_typeStats()

how about a summary plot?

In [ ]:
def plot_typeStats(populations):
    """
    
    """
    if isinstance(populations, list):
        stats = {p.name : p.get_typeStats() for p in populations}
        fig, ax = plt.subplots()
        stellarMasses = [0.3, 0.5, 0.7, 1.0]
        for M, Mstar in enumerate(stats.keys()):
            for planetType in stats[Mstar]:
                plt.errorbar(stellarMasses[M], stats[Mstar][planetType]['meanMetallicity'], stats[Mstar][planetType]['stdMetallicity'],
                             capsize=40., fmt='o', label=planetType)
        plt.legend()

## disk lifetime analysis

In [ ]:
tDiskFiles = [outputs + s for s in ['ref_redtdisk_0.3Msol.dat','ref_redtdisk_0.5Msol.dat', 'ref_redtdisk_0.7Msol.dat', 'ref_redtdisk_1.0Msol.dat']]

In [ ]:
Mgrid = output.Population()
Mgrid.read_tDiskData(tDiskFiles)

compute disk fractions and store them separately for each Mstar

In [ ]:
diskFractions = {Mstar : utils.get_diskFractions(tDiskData, Ntimes=50) for Mstar, tDiskData in Mgrid.tDiskData.groupby(level=0)}

fit exponential function to disk fractions

To parametrize the fraction of remaining disks, we apply a simple exponential fit of the form f(t) = A*exp(-t/tau) + C. There is a clear trend of increasing disk lifetimes with increasing stellar mass. 

In [ ]:
import ipdb; ipdb.set_trace()

In [ ]:
ax, diskFractions, tau, std_tau = plots.plot_diskFractions(diskFractions)

what is the trend in 'tau'?

In [ ]:
Mstar = [float(key[:-4]) for key in diskFractions.keys()]
# tau = [m[2][1] for m in diskFractions.values()]

In [ ]:
ax = plots.plot_diskLifetimeComparison(Mstar, tau, std_tau, nSigma=1)

In this plot we see that the scaling of tau is positive and linear in host star mass. This contradicts the findings of Ribas et al. (2015), who not only find a negative trend with some significance, but also report typical infrared excess decay times of ~(2-6)Myr, which is consistently lower than our values

### Mstar-dependent histograms

In [ ]:
def plot_multiHistogram(dataFrame, columnNames, ax=None, **kwargs):
    """ Plot multiple histograms from a dataFrame into the same axis.
    
    Parameters
    ----------
    dataFrame : pandas DataFrame
        dataFrame containing column(s) stated in 'columnNames'
    columnNames : str or list
        name(s) of the column(s) to plot.
        
    Returns
    -------
    ax : matplotlib axis
        axis with the plot
    """
    if ax == None:
        fig, ax = plt.subplots()
    
    if not kwargs:
        mplKwargs = {'alpha' : 0.6,
                     'histtype' : 'stepfilled',
                     'edgecolor' : 'black',
                     'linewidth' : 1.2,
                     'bins' : 50}
    else:
        mplKwargs = {}
    
    if isinstance(columnNames, list):
        # plot different columns of the same dataFrame        
        for name in columnNames:
            ax.hist(dataFrame[name], label=name, **mplKwargs, **kwargs)
    
    elif isinstance(columnNames, str):
        # assume that dataFrame is multiindexed, iterate over outermost level
        for label, subpopulation in dataFrame.groupby(level=0):
           ax.hist(subpopulation[columnNames], label=label, **mplKwargs, **kwargs)
    ax.legend()
    return ax


# test
# group = Mgrid.tDiskData.groupby(level=0)
# tDiskData03 = group.get_group('0.3Msol')
# plot_multiHistogram(tDiskData03, ['m', 'r', 't'], bins=50)

# more tests
# plot_multiHistogram(Mgrid.tDiskData, 'm', bins=50)

## create occurrence maps

In [ ]:
plots.plot_occurrence?

#### plot all maps into one figure

In [ ]:
# fix all ranges for better comparison of subpopulations
smaRange = [1e-2, 50.] 
radRange = [2e-1, 400.]
colRange = [0., 10.]

fig, axes = plt.subplots(2,2, sharex=True, sharey=True)

for (label, population), ax in zip(Mgrid.tDiskData.groupby(level=0), axes.reshape(-1)):

    h, x, y, ax, im = plots.plot_occurrence(population, xAxis='a', logColormap=True, xRange=smaRange,
                                        yRange=radRange, zRange=colRange, showColorbar=False, ax=ax)
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    t = ax.text(.07, .85, label[:-4] + ' M$_\odot$', color='white', transform=ax.transAxes)

    
fig.text(0.45, 0.01, 'Semimajor Axis [au]', ha='center')
fig.text(0.01, 0.55, 'Planet Size [$\mathrm{R_{Earth}}$]', va='center', rotation='vertical')
  
cbar_ax = fig.add_axes([0.84, 0.13, 0.04, 0.84])
cbarlabel = r"Planets per 100 Stars per $a-R_P$ interval"
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label(cbarlabel, labelpad=5.)

plt.subplots_adjust(left=.11, bottom=.13, right=.79, top=.97, wspace=.08, hspace=.08)

## Joint simlist - population analysis

In [ ]:
# read simulation lists into population
simlistFiles = [outputs + s for s in ['simulation_list_0.3Msol.dat', 'simulation_list_0.5Msol.dat', 'simulation_list_0.7Msol.dat', 'simulation_list_1.0Msol.dat']]
Mgrid.read_simlist(simlistFiles)

### check scaling of initial disk mass

In [ ]:
%pylab

In [ ]:
ax = plot_multiHistogram(Mgrid.simlist, 'sigma0', histtype='stepfilled',
                    edgecolor='black', linewidth=1.2, bins=50)

In [ ]:
Mgrid.simlist.groupby(level=0).sigma0.describe()

translate to disk mass to check its distribution (we have to work with a smaller sample because 400'000 entries are a bit heavy)

In [ ]:
Mgrid.simlist['mDisk'] = Mgrid.simlist['sigma0']

simlistSample= Mgrid.simlist.sample(10000)

In [ ]:
simlistSample['mDisk'] = simlistSample['mDisk'].apply(utils.get_M0, args=(simlistSample['a_out'], 0.9))

In [ ]:
simlistSample.groupby(level=0).mDisk.describe()

In [ ]:
plot_multiHistogram(simlistSample, 'mDisk')#, histtype='stepfilled', edgecolor='black', linewidth=1.2, bins=50)

In [ ]:
simlistSample.groupby(level=0).groups.keys()

does the linear scaling function work?

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(0,10)
y = utils.linearScale(0,10,2,10,x)
ax.plot(x, y)
plt.show()

looks good. 

### check relation between Tdisk, Mwind, Sigma0

In [ ]:
from scipy.optimize import curve_fit

def tDiskRel(Mwind, Sigma0, a, b, c):
    return a*(1e-7/Mwind)**b*(Sigma0/100)**c

params, covariance = curve_fit(tDiskRel, [simlistSample['windM'], simlistSample['sigma0']])

In [ ]:
params

In [ ]:
plt.plot(simlistSample['windM'], tDiskRel(simlistSample['windM'], simlistSample['sigma0'], *params))